# Test NotionDB connection
ref: https://python.langchain.com/docs/integrations/document_loaders/notiondb

In [15]:
from langchain_community.document_loaders import NotionDBLoader

In [3]:
import tomllib

In [12]:
with open('../.tokens.toml', 'rb') as f:
    _TOKEN_NOTION = tomllib.load(f)['notion']

with open('../.notion_databases.toml', 'rb') as f:
    _DATABASES_NOTION = tomllib.load(f)

In [14]:
_DATABASES_NOTION

{'笔记（非文学）': '7443174d151342458fb7e47acdbb0c64',
 '读书笔记（文学）': '808e50735cce4c66a151f3e8b79c8d9d',
 '写作': '8b81c07089344d8d95ddba22bafb8c12',
 '格言小集': '04d75f24b9544f3f8b220d5e7d87b7c3'}

In [18]:
loader = NotionDBLoader(
    integration_token=_TOKEN_NOTION,
    database_id=_DATABASES_NOTION['格言小集'],
    request_timeout_sec=30,  # optional, defaults to 10
)

In [20]:
%%time
docs = loader.load()

CPU times: user 268 ms, sys: 21.9 ms, total: 290 ms
Wall time: 5.87 s


In [24]:
print(docs[:2])

[Document(page_content='人生四不捡：塔吊下边冰红茶，铁轨上边牛肉干，过山车下八宝粥，课桌后边葡萄干', metadata={'name': '笑死', 'id': '2303056b-72f1-4f9a-b377-cf6b12a0d424'}), Document(page_content='\n名为太阳的溶剂\n\n楼群之中\n乌鸦堕落为一种失去预言性的可厌生物\n\n对大多数人来说，他们所抱定的观念只是为他们生存方式做辩护的使用说明书而已\n\n颤抖的彩虹其实是光的边角余料。\n\n以前的建筑是不透明的甲壳类，现在的建筑是透光的腔肠类。\n\n射电望远镜就是宇宙射线的向日葵。\n\n电线像静脉一样生长着。\n\n绝望的尽头并非对绝望感到绝望，而是对绝望感到厌烦。相对地，希望的尽头也只是对希望的疲倦而已。与一般的见解不同，绝望和希望是生活的激发态。\n', metadata={'name': '沃滋集朔德', 'id': '340f4781-1042-4534-b286-b758b0ca09a4'})]
